In [1]:
import os
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters
import dialogflow
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

updater = Updater(token='1076199008:AAEVFbrhs-K7QOQw3p9ap-TkLByL4pQN3ZM') # Токен API к Telegram
dispatcher = updater.dispatcher
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/home/alex/Downloads/small-talk-gesu-809b2c029117.json'# скачнный JSON


DIALOGFLOW_PROJECT_ID = 'small-talk-gesu' #PROJECT ID из DialogFlow 
DIALOGFLOW_LANGUAGE_CODE = 'ru' # язык
SESSION_ID = 'gb_project_bot'  # ID бота из телеграма

In [2]:
question = None
written = False

with open("/home/alex/Downloads/nlp/prepared_answers.txt", "w") as fout:
    with open("/home/alex/Downloads/nlp/Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

In [3]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [4]:
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("/home/alex/Downloads/nlp/Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 100000:
            break

KeyboardInterrupt: 

In [ ]:
sentences = [i for i in sentences if len(i) > 2]

In [ ]:
modelFT = FastText(sentences=sentences, size=200, min_count=2, window=5)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def identity_tokenizer(text):
    return text

In [ ]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=identity_tokenizer,
                                   lowercase=False, max_df=0.9)

In [ ]:
tfidf_vectorizer.fit_transform(sentences)

In [ ]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(200)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += tfidf_vectorizer.idf_[tfidf_vectorizer.vocabulary_[word]]
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3)
    return [index_map[i] for i in answers]

In [ ]:
def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text='Добрый день')

def textMessage(bot, update):

    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(DIALOGFLOW_PROJECT_ID, SESSION_ID)
    text_input = dialogflow.types.TextInput(text=update.message.text , language_code=DIALOGFLOW_LANGUAGE_CODE)
    query_input = dialogflow.types.QueryInput(text=text_input)
    try:
        response = session_client.detect_intent(session=session, query_input=query_input)
    except InvalidArgument:
         raise

    text = response.query_result.fulfillment_text
    if text:
        bot.send_message(chat_id=update.message.chat_id, text= response.query_result.fulfillment_text)
    else:
        custom_response = get_response(update.message.text, ft_index, modelFT, index_map)
        bot.send_message(chat_id=update.message.chat_id, text=custom_response[0])


In [ ]:
# Хендлеры
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
# Добавляем хендлеры в диспетчер
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
# Начинаем поиск обновлений
updater.start_polling(clean=True)
# Останавливаем бота, если были нажаты Ctrl + C
updater.idle()